# **Lab 10: Ιmplementing BB84 - Part II**
---

### **Description**
In today's lab, we will explore the role of Eve in BB84.

<br>

### **Structure**
**Part 1**: [Alice and Bob](#p1)

**Part 2**: [Alice, Bob, and Eve's Measurement Attack](#p2)

**Part 3**: [Alice, Bob, and Eve's Intercept and Resent Attack](#p3)

**Part 4**: [Alice, Bob, and Eve's Entanglement Attack](#p4)



<br>

### **Learning Objectives**
By the end of this lab, you will:

* Recognize how to implement BB84 in cirq, particularly including a measurement attack by Eve.

* Recognize how to adjust Eve’s attack in BB84 in cirq.

<br>

### **Resources**
* [BB84 Cheat Sheet](https://docs.google.com/document/d/1FTBVWQsRVPvuP5e4lo3D62F0NOyfA1qIIPrDnocV6nc/edit)

<br>

**Before starting, run the code below to import all necessary functions and libraries.**

In [ ]:
!pip install cirq --quiet
import cirq
from random import choices
import binascii

<a name="p1"></a>

---
## **Part 1: Alice and Bob**
---

The entire BB84 protocol between Alice and Bob is implemented below.

---
### **Part 1.1: Setup**
---

#### **Problem #1.1.1**

**Together**, let's define a dictionary called `encode_gates` that specify which gates to apply based on the bit value. Then, let's define a dictionary called `basis_gates` that specify which gates to apply based on the basis. Finally, let's create a list of `NamedQubit`s that is `num_bits` long and has the prefix `q`.

In [ ]:
encode_gates = {0: # COMPLETE THIS CODE
basis_gates = {'Z': # COMPLETE THIS CODE

num_bits = 10
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

#####**Solution**

In [ ]:
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}

num_bits = 10
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

---
### **Part 1.2: Implementing the Steps**
---

---
#### **Phase #1: Alice Sends**
---

#### **Step #1: Alice Randomly Chooses Bits**

**Together**, let's use python's `choices(...)` function to create Alice's key of random bits that is `num_bits` long.

In [ ]:
alice_key = choices(# COMPLETE THIS CODE

print('Alice\'s initial key: ', alice_key)

#####**Solution**

In [ ]:
alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

Alice's initial key:  [0, 0, 0, 0, 0, 0, 1, 0, 1, 0]


<a name="s2"></a>

#### **Step #2: Alice Randomly Chooses Bases**

**Together**, let's use python's `choices(...)` function to create Alice's `num_bits` basis choices.

In [ ]:
alice_bases = choices(# COMPLETE THIS CODE

print('\nAlice\'s randomly chosen bases: ', alice_bases)

#####**Solution**

In [ ]:
alice_bases = choices(['Z', 'X'], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)


Alice's randomly chosen bases:  ['X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X']


<a name="s3"></a>

#### **Step #3: Alice Creates Qubits**

**Together**, create Alice's qubits based on her choice of bit and basis. Complete the code below so that the appropriate gates are appended to `alice_circuit` within the loop.

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(# COMPLETE THIS CODE

  encode_value = alice_key[bit]
  encode_gate = encode_gates[# COMPLETE THIS CODE

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[# COMPLETE THIS CODE

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(# COMPLETE THIS CODE
  alice_circuit.append(basis_gate(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

<a name="s4"></a>

#### **Step #4: Alice Sends the Qubits to Bob**

This step doesn't require us to do anything in Python. However, in real life, this would be where Alice sends Bob the qubits through a public quantum channel.

---
#### **Phase #2: Bob Receives**
---

<a name="s5"></a>

#### **Step #5: Bob Randomly Chooses Bases**

**Independently**, randomly choose Bob's `num_bits` bases and apply the appropriate gates to the qubits he received, `qubits`. Complete the code below to accomplish this for each qubit.

In [ ]:
bob_bases = choices(# COMPLETE THIS CODE


bob_circuit = cirq.Circuit()

for # COMPLETE THIS CODE

  basis_value = # COMPLETE THIS CODE
  basis_gate = # COMPLETE THIS CODE

  qubit = # COMPLETE THIS CODE
  bob_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
bob_bases = choices(['Z', 'X'], k = num_bits)
print('Bob\'s randomly chosen bases: ', bob_bases)

bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

Bob's randomly chosen bases:  ['X', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X']


<a name="s6"></a>

#### **Step #6: Bob Measures the Qubits**

**Independently**, make a measurement of all of the qubits for Bob. Complete the code below to append this action to `bob_circuit`. Ensure the measure method is given the parameter `key = 'bob key'` so that we can easily retrieve this result later.

In [ ]:
bob_circuit.append(# COMPLETE THIS CODE

print(bob_circuit)

#####**Solution**

In [ ]:
bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

print(bob_circuit)

q0: ───H───M('bob key')───
           │
q1: ───H───M──────────────
           │
q2: ───H───M──────────────
           │
q3: ───H───M──────────────
           │
q4: ───H───M──────────────
           │
q5: ───H───M──────────────
           │
q6: ───I───M──────────────
           │
q7: ───I───M──────────────
           │
q8: ───H───M──────────────
           │
q9: ───H───M──────────────


<a name="s7"></a>

#### **Step #7: Bob Creates a Key**

**Independently**, create a key for Bob from the mesurement result of each qubit.

<br>

**Run the code below to accomplish this step.**

In [ ]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)

#####**Solution**

In [ ]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [0 1 0 1 1 0 1 0 1 0]


---
#### **Phase #3: Alice and Bob Compare**
---

<a name="s8"></a>

#### **Step #8: Alice and Bob Compare Bases**

**Independently**, compare Alice's and Bob's randomly selected bases.

**Run the code below to accomplish this step.**

In [ ]:
final_alice_key = []
final_bob_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)

#####**Solution**

In [ ]:
final_alice_key = []
final_bob_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)


Alice's key:  [0, 0, 1, 0, 1, 0]
Bob's key:  [0, 0, 1, 0, 1, 0]


<a name="s9"></a>

#### **Step #9: Alice and Bob Compare the First Bits in Their Key**

**Independently**, compare the first few bits in Alice's and Bob's key to ensure the protocol was successful.

<br>

**Run the code below to accomplish this.**

In [ ]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')

#####**Solution**

In [ ]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



We can use our keys!
Alice Key:  [0, 1, 0]
Bob Key:  [0, 1, 0]


<a name="p2"></a>

---
## **Part 2: Alice, Bob, and Eve's Measurement Attack**
---

Now, we will modify this code to introduce Eve, who will be performing a measurement attack.

### **Part 2.1: The Setup**

#### **Problem #2.1.1**

**Together**, let's define a dictionary called `encode_gates` that specify which gates to apply based on the bit value. Then, let's define a dictionary called `basis_gates` that specify which gates to apply based on the basis. Finally, let's create a list of `NamedQubit`s that is `num_bits` long and has the prefix `q`.

In [ ]:
encode_gates = {0: # COMPLETE THIS CODE
basis_gates = {'Z': # COMPLETE THIS CODE

num_bits = 10
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

#####**Solution**

In [ ]:
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}

num_bits = 100
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

### **Part 2.2: Implementing the Steps**

---
#### **Phase #1: Alice Sends**
---

<a name="s1"></a>

#### **Step #1: Alice Randomly Chooses Bits**

**Together**, let's use python's `choices(...)` function to create Alice's key of random bits that is `num_bits` long.

In [ ]:
alice_key = choices(# COMPLETE THIS CODE

print('Alice\'s initial key: ', alice_key)

#####**Solution**

In [ ]:
alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

Alice's initial key:  [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0]


<a name="s2"></a>

#### **Step #2: Alice Randomly Chooses Bases**

**Together**, let's use python's `choices(...)` function to create Alice's `num_bits` basis choices.

In [ ]:
alice_bases = choices(# COMPLETE THIS CODE

print('\nAlice\'s randomly chosen bases: ', alice_bases)

#####**Solution**

In [ ]:
alice_bases = choices(['Z', 'X'], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)


Alice's randomly chosen bases:  ['Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z']


<a name="s3"></a>

#### **Step #3: Alice Creates Qubits**

**Independently**, create Alice's qubits based on her choice of bit and basis. Complete the code below so that the appropriate gates are appended to `alice_circuit` within the loop.

In [ ]:
alice_circuit = cirq.Circuit()

for # COMPLETE THIS CODE

  encode_value = alice_key[# COMPLETE THIS CODE
  encode_gate = encode_gates[# COMPLETE THIS CODE

  basis_value = alice_bases[# COMPLETE THIS CODE
  basis_gate = basis_gates[# COMPLETE THIS CODE

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

#####**Solution**

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

<a name="s4"></a>

#### **Step #4: Alice Sends the Qubits to Bob**

This step doesn't require us to do anything in Python. However, in real life, this would be where Alice sends Bob the qubits through a public quantum channel.

---
#### **Phase #2: Eve Receives**
---

<a name="s4"></a>

#### **Step #5: Eve Randomly Chooses Bases**

For this attack, Eve does not try to guess bases, so no code is required.

#### **Step #6: Eve Measures the Qubits**
**Independently**, make a measurement of all of the qubits for Eve. Complete the code below to append this action to `eve_circuit`. Ensure the measure method is given the parameter `key = 'eve key'` so that we can easily retrieve this result later.

In [ ]:
eve_circuit = cirq.Circuit()
eve_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
eve_circuit = cirq.Circuit()
eve_circuit.append(cirq.measure(qubits, key = 'eve key'))

#### **Step #7: Eve Creates a Key**

**Independently**, create a key for Eve from the mesurement result of each qubit.

<br>

**Run the code below to accomplish this step.**

In [ ]:
eve_intercept_circuit = alice_circuit + eve_circuit

sim = cirq.Simulator()
results = sim.run(eve_intercept_circuit)
eve_key = results.measurements['eve key'][0]

print('\nEve\'s initial key: ', eve_key)

#####**Solution**

In [ ]:
eve_intercept_circuit = alice_circuit + eve_circuit

sim = cirq.Simulator()
results = sim.run(eve_intercept_circuit)
eve_key = results.measurements['eve key'][0]

print('\nEve\'s initial key: ', eve_key)


Eve's initial key:  [0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1
 1 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 0 0 1 0 1 0 1
 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1 0 0]


#### **Eve Repeats Step #3 to Fool Bob**

**Run the code below to accomplish this step.**

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = eve_key[bit]
  encode_gate = encode_gates[encode_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))

print('\nAlice\'s Phase 1 circuit after Eve\'s interception:\n', alice_circuit)

#####**Solution**

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = eve_key[bit]
  encode_gate = encode_gates[encode_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))

print('\nAlice\'s Phase 1 circuit after Eve\'s interception:\n', alice_circuit)


Alice's Phase 1 circuit after Eve's interception:
 q0: ────I───

q1: ────I───

q2: ────I───

q3: ────I───

q4: ────X───

q5: ────X───

q6: ────X───

q7: ────X───

q8: ────X───

q9: ────X───

q10: ───X───

q11: ───I───

q12: ───I───

q13: ───X───

q14: ───X───

q15: ───X───

q16: ───X───

q17: ───I───

q18: ───I───

q19: ───X───

q20: ───X───

q21: ───I───

q22: ───X───

q23: ───X───

q24: ───X───

q25: ───X───

q26: ───X───

q27: ───I───

q28: ───I───

q29: ───I───

q30: ───X───

q31: ───X───

q32: ───I───

q33: ───I───

q34: ───X───

q35: ───X───

q36: ───X───

q37: ───X───

q38: ───I───

q39: ───I───

q40: ───X───

q41: ───X───

q42: ───I───

q43: ───X───

q44: ───X───

q45: ───I───

q46: ───I───

q47: ───X───

q48: ───X───

q49: ───I───

q50: ───X───

q51: ───I───

q52: ───X───

q53: ───X───

q54: ───X───

q55: ───I───

q56: ───I───

q57: ───X───

q58: ───X───

q59: ───X───

q60: ───X───

q61: ───I───

q62: ───X───

q63: ───I───

q64: ───I───

q65: ───X───

q66: ───X───

q67: ───I─

---
#### **Phase #2: Bob Receives**
---

<a name="s5"></a>

#### **Step #5: Bob Randomly Chooses Bases**

**Independently**, randomly choose Bob's `num_bits` bases and apply the appropriate gates to the qubits he received, `qubits`. Complete the code below to accomplish this for each qubit.

In [ ]:
bob_bases = choices(# COMPLETE THIS CODE


bob_circuit = cirq.Circuit()

for # COMPLETE THIS CODE

  basis_value = # COMPLETE THIS CODE
  basis_gate = # COMPLETE THIS CODE

  qubit = # COMPLETE THIS CODE
  bob_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
bob_bases = choices(['Z', 'X'], k = num_bits)
print('Bob\'s randomly chosen bases: ', bob_bases)

bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

Bob's randomly chosen bases:  ['Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X']


<a name="s6"></a>

#### **Step #6: Bob Measures the Qubits**

**Independently**, make a measurement of all of the qubits for Bob. Complete the code below to append this action to `bob_circuit`. Ensure the measure method is given the parameter `key = 'bob key'` so that we can easily retrieve this result later.

In [ ]:
bob_circuit.append(# COMPLETE THIS CODE

print(bob_circuit)

#####**Solution**

In [ ]:
bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

print(bob_circuit)

q0: ────I───M('bob key')───
            │
q1: ────I───M──────────────
            │
q2: ────I───M──────────────
            │
q3: ────I───M──────────────
            │
q4: ────I───M──────────────
            │
q5: ────I───M──────────────
            │
q6: ────I───M──────────────
            │
q7: ────I───M──────────────
            │
q8: ────H───M──────────────
            │
q9: ────H───M──────────────
            │
q10: ───H───M──────────────
            │
q11: ───I───M──────────────
            │
q12: ───I───M──────────────
            │
q13: ───I───M──────────────
            │
q14: ───H───M──────────────
            │
q15: ───H───M──────────────
            │
q16: ───H───M──────────────
            │
q17: ───I───M──────────────
            │
q18: ───H───M──────────────
            │
q19: ───H───M──────────────
            │
q20: ───I───M──────────────
            │
q21: ───H───M──────────────
            │
q22: ───I───M──────────────
            │
q23: ───I───M──────────────
      

<a name="s7"></a>

#### **Step #7: Bob Creates a Key**

**Independently**, create a key for Bob from the mesurement result of each qubit.

<br>

**Run the code below to accomplish this step.**

In [ ]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)

#####**Solution**

In [ ]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [0 0 0 0 1 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 0 1 1 1
 0 0 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0
 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 1 1 0 0 1]


---
#### **Phase #3: Alice and Bob Compare**
---

<a name="s8"></a>

#### **Step #8: Alice and Bob Compare Bases**

**Independently**, compare Alice's and Bob's randomly selected bases. Complete the code below to accomplish this.

**Run the code below to accomplish this step.**

In [ ]:
final_alice_key = []
final_bob_key = []
final_eve_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])
    final_eve_key.append(eve_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)

#####**Solution**

In [ ]:
final_alice_key = []
final_bob_key = []
final_eve_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])
    final_eve_key.append(eve_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)


Alice's key:  [0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0]
Bob's key:  [0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0]
Eve's key:  [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0]


<a name="s9"></a>

#### **Step #9: Alice and Bob Compare the First Bits in Their Key**

**Independently**, compare the first few bits in Alice's and Bob's key to ensure the protocol was successful.

<br>

**Run the code below to accomplish this.**

In [ ]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')

#####**Solution**

In [ ]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



Eve was listening, we need to use a different channel!


<a name="p3"></a>

---
## **Part 3: Alice, Bob, and Eve's Intercept and Resend Attack**
---

Now, we will modify this code so that Eve performs an Intercept and Resend attack.

### **Part 3.1: The Setup**

#### **Problem #3.1.1**

**Independently**, let's define a dictionary called `encode_gates` that specify which gates to apply based on the bit value. Then, let's define a dictionary called `basis_gates` that specify which gates to apply based on the basis. Finally, let's create a list of `NamedQubit`s that is `num_bits` long and has the prefix `q`.

In [ ]:
encode_gates = {# COMPLETE THIS CODE
basis_gates = {# COMPLETE THIS CODE

num_bits = 100
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

#####**Solution**

In [ ]:
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}

num_bits = 100
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

### **Part 3.2: Implementing the Steps**

---
#### **Phase #1: Alice Sends**
---

<a name="s1"></a>

#### **Step #1: Alice Randomly Chooses Bits**

**Independently**, let's use python's `choices(...)` function to create Alice's key of random bits that is `num_bits` long.

In [ ]:
alice_key = choices(# COMPLETE THIS CODE

print('Alice\'s initial key: ', alice_key)

#####**Solution**

In [ ]:
alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

Alice's initial key:  [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0]


<a name="s2"></a>

#### **Step #2: Alice Randomly Chooses Bases**

**Independently**, let's use python's `choices(...)` function to create Alice's `num_bits` basis choices.

In [ ]:
alice_bases = choices(# COMPLETE THIS CODE

print('\nAlice\'s randomly chosen bases: ', alice_bases)

#####**Solution**

In [ ]:
alice_bases = choices(['Z', 'X'], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)


Alice's randomly chosen bases:  ['Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z']


<a name="s3"></a>

#### **Step #3: Alice Creates Qubits**

**Independently**, create Alice's qubits based on her choice of bit and basis. Complete the code below so that the appropriate gates are appended to `alice_circuit` within the loop.

In [ ]:
alice_circuit = cirq.Circuit()

for # COMPLETE THIS CODE

  encode_value = # COMPLETE THIS CODE
  encode_gate = # COMPLETE THIS CODE

  basis_value = # COMPLETE THIS CODE
  basis_gate = # COMPLETE THIS CODE

  qubit = # COMPLETE THIS CODE
  alice_circuit.append(# COMPLETE THIS CODE
  alice_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

<a name="s4"></a>

#### **Step #4: Alice Sends the Qubits to Bob**

This step doesn't require us to do anything in Python. However, in real life, this would be where Alice sends Bob the qubits through a public quantum channel.

---
#### **Phase #2: Eve Receives**
---

#### **Step #5: Eve Randomly Chooses Bases**

**Independently**, randomly choose Eve's `num_bits` bases and apply the appropriate gates to the qubits she received, `qubits`. Complete the code below to accomplish this for each qubit.

In [ ]:
eve_bases = choices(# COMPLETE THIS CODE
print('Eve\'s randomly chosen bases: ', eve_bases)

eve_circuit = cirq.Circuit()

for # COMPLETE THIS CODE
  basis_value = # COMPLETE THIS CODE
  basis_gate = # COMPLETE THIS CODE

  qubit = # COMPLETE THIS CODE
  eve_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
eve_bases = choices(['Z', 'X'], k = num_bits)
print('Eve\'s randomly chosen bases: ', eve_bases)

eve_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = eve_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  eve_circuit.append(basis_gate(qubit))

Eve's randomly chosen bases:  ['Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z']


#### **Step #6: Eve Measures the Qubits**
**Independently**, make a measurement of all of the qubits for Eve. Complete the code below to append this action to `eve_circuit`. Ensure the measure method is given the parameter `key = 'eve key'` so that we can easily retrieve this result later.

In [ ]:
eve_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
eve_circuit.append(cirq.measure(qubits, key = 'eve key'))

#### **Step #7: Eve Creates a Key**

**Independently**, create a key for Eve from the mesurement result of each qubit.

<br>

**Run the code below to accomplish this step.**

In [ ]:
eve_intercept_circuit = alice_circuit + eve_circuit

sim = cirq.Simulator()
results = sim.run(eve_intercept_circuit)
eve_key = results.measurements['eve key'][0]

print('\nEve\'s initial key: ', eve_key)

#####**Solution**

In [ ]:
eve_intercept_circuit = alice_circuit + eve_circuit

sim = cirq.Simulator()
results = sim.run(eve_intercept_circuit)
eve_key = results.measurements['eve key'][0]

print('\nEve\'s initial key: ', eve_key)


Eve's initial key:  [0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1
 1 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 0 0 1 0 1 0 1
 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1 0 0]


#### **Eve Repeats Step #3 to Fool Bob**

**Run the code below to accomplish this step.**

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = eve_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = eve_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

print('\nAlice\'s Phase 1 circuit after Eve\'s interception:\n', alice_circuit)

#####**Solution**

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = eve_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = eve_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

print('\nAlice\'s Phase 1 circuit after Eve\'s interception:\n', alice_circuit)


Alice's Phase 1 circuit after Eve's interception:
 q0: ────I───I───

q1: ────I───I───

q2: ────I───I───

q3: ────I───I───

q4: ────X───I───

q5: ────X───I───

q6: ────X───H───

q7: ────X───H───

q8: ────X───H───

q9: ────X───H───

q10: ───X───I───

q11: ───I───H───

q12: ───I───H───

q13: ───X───I───

q14: ───X───I───

q15: ───X───I───

q16: ───X───H───

q17: ───I───H───

q18: ───I───H───

q19: ───X───I───

q20: ───X───I───

q21: ───I───I───

q22: ───X───I───

q23: ───X───H───

q24: ───X───H───

q25: ───X───H───

q26: ───X───H───

q27: ───I───I───

q28: ───I───H───

q29: ───I───I───

q30: ───X───I───

q31: ───X───H───

q32: ───I───I───

q33: ───I───H───

q34: ───X───H───

q35: ───X───I───

q36: ───X───H───

q37: ───X───I───

q38: ───I───H───

q39: ───I───H───

q40: ───X───I───

q41: ───X───H───

q42: ───I───H───

q43: ───X───I───

q44: ───X───H───

q45: ───I───I───

q46: ───I───H───

q47: ───X───H───

q48: ───X───I───

q49: ───I───H───

q50: ───X───H───

q51: ───I───H───

q52: ───X───

---
#### **Phase #2: Bob Receives**
---

<a name="s5"></a>

#### **Step #5: Bob Randomly Chooses Bases**

**Independently**, randomly choose Bob's `num_bits` bases and apply the appropriate gates to the qubits he received, `qubits`. Complete the code below to accomplish this for each qubit.

In [ ]:
bob_bases = choices(# COMPLETE THIS CODE


bob_circuit = cirq.Circuit()

for # COMPLETE THIS CODE

  basis_value = # COMPLETE THIS CODE
  basis_gate = # COMPLETE THIS CODE

  qubit = # COMPLETE THIS CODE
  bob_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
bob_bases = choices(['Z', 'X'], k = num_bits)
print('Bob\'s randomly chosen bases: ', bob_bases)

bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

Bob's randomly chosen bases:  ['Z', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z']


<a name="s6"></a>

#### **Step #6: Bob Measures the Qubits**

**Independently**, make a measurement of all of the qubits for Bob. Complete the code below to append this action to `bob_circuit`. Ensure the measure method is given the parameter `key = 'bob key'` so that we can easily retrieve this result later.

In [ ]:
bob_circuit.append(# COMPLETE THIS CODE

print(bob_circuit)

#####**Solution**

In [ ]:
bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

print(bob_circuit)

q0: ────I───M('bob key')───
            │
q1: ────H───M──────────────
            │
q2: ────H───M──────────────
            │
q3: ────H───M──────────────
            │
q4: ────H───M──────────────
            │
q5: ────H───M──────────────
            │
q6: ────H───M──────────────
            │
q7: ────I───M──────────────
            │
q8: ────H───M──────────────
            │
q9: ────H───M──────────────
            │
q10: ───I───M──────────────
            │
q11: ───H───M──────────────
            │
q12: ───I───M──────────────
            │
q13: ───H───M──────────────
            │
q14: ───H───M──────────────
            │
q15: ───I───M──────────────
            │
q16: ───I───M──────────────
            │
q17: ───I───M──────────────
            │
q18: ───H───M──────────────
            │
q19: ───H───M──────────────
            │
q20: ───I───M──────────────
            │
q21: ───I───M──────────────
            │
q22: ───H───M──────────────
            │
q23: ───H───M──────────────
      

<a name="s7"></a>

#### **Step #7: Bob Creates a Key**

**Independently**, create a key for Bob from the mesurement result of each qubit.

<br>

**Run the code below to accomplish this step.**

In [ ]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)

#####**Solution**

In [ ]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [0 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 0
 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 0 1
 1 0 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1 0 0]


---
#### **Phase #3: Alice and Bob Compare**
---

<a name="s8"></a>

#### **Step #8: Alice and Bob Compare Bases**

**Independently**, compare Alice's and Bob's randomly selected bases.

**Run the code below to accomplish this step.**

In [ ]:
final_alice_key = []
final_bob_key = []
final_eve_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])
    final_eve_key.append(eve_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)

#####**Solution**

In [ ]:
final_alice_key = []
final_bob_key = []
final_eve_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])
    final_eve_key.append(eve_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)


Alice's key:  [0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0]
Bob's key:  [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]
Eve's key:  [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]


<a name="s9"></a>

#### **Step #9: Alice and Bob Compare the First Bits in Their Key**

**Independently**, compare the first few bits in Alice's and Bob's key to ensure the protocol was successful.

<br>

**Run the code below to accomplish this.**

In [ ]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')

#####**Solution**

In [ ]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



Eve was listening, we need to use a different channel!


<a name="p4"></a>

---
## **Part 4: Alice, Bob, and Eve's Entanglement Attack**
---

Now, we will modify this code so that Eve performs an Entanglement attack.

### **Part 4.1: The Setup**

#### **Problem #4.1.1**

**Independently**, define a dictionary called `encode_gates` that specify which gates to apply based on the bit value. Then, let's define a dictionary called `basis_gates` that specify which gates to apply based on the basis. Finally, let's create a list of `NamedQubit`s that is `num_bits` long and has the prefix `q`.

In [ ]:
encode_gates = {# COMPLETE THIS CODE
basis_gates = {# COMPLETE THIS CODE

num_bits = 100
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

#####**Solution**

In [ ]:
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}

num_bits = 100
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

---
#### **Phase #1: Alice Sends**
---

<a name="s1"></a>

#### **Step #1: Alice Randomly Chooses Bits**

**Together**, let's use python's `choices(...)` function to create Alice's key of random bits that is `num_bits` long.

In [ ]:
alice_key = choices(# COMPLETE THIS CODE

print('Alice\'s initial key: ', alice_key)

#####**Solution**

In [ ]:
alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

Alice's initial key:  [1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0]


<a name="s2"></a>

#### **Step #2: Alice Randomly Chooses Bases**

**Together**, let's use python's `choices(...)` function to create Alice's `num_bits` basis choices.

In [ ]:
alice_bases = choices(# COMPLETE THIS CODE

print('\nAlice\'s randomly chosen bases: ', alice_bases)

#####**Solution**

In [ ]:
alice_bases = choices(['Z', 'X'], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)


Alice's randomly chosen bases:  ['Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z']


<a name="s3"></a>

#### **Step #3: Alice Creates Qubits**

**Together**, create Alice's qubits based on her choice of bit and basis. Complete the code below so that the appropriate gates are appended to `alice_circuit` within the loop.

In [ ]:
alice_circuit = cirq.Circuit()

for # COMPLETE THIS CODE

  encode_value = # COMPLETE THIS CODE
  encode_gate = # COMPLETE THIS CODE

  basis_value = # COMPLETE THIS CODE
  basis_gate = # COMPLETE THIS CODE

  qubit = # COMPLETE THIS CODE
  alice_circuit.append(# COMPLETE THIS CODE
  alice_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

<a name="s4"></a>

#### **Step #4: Alice Sends the Qubits to Bob**

This step doesn't require us to do anything in Python. However, in real life, this would be where Alice sends Bob the qubits through a public quantum channel.

---
#### **Phase #2: Eve Receives**
---

#### **Step #5: Eve Randomly Entangles Her Qubits**

**Independently**, create `num_bits` qubits with the prefix "eve" for Eve and entangle them with Alice's sent qubits.

In [ ]:
eve_qubits = # COMPLETE THIS CODE

for # COMPLETE THIS CODE

  qubit = # COMPLETE THIS CODE
  eve_qubit = # COMPLETE THIS CODE

  alice_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
eve_qubits = cirq.NamedQubit.range(num_bits, prefix="eve")

for bit in range(num_bits):

  qubit = qubits[bit]
  eve_qubit = eve_qubits[bit]

  alice_circuit.append(cirq.CNOT(qubit, eve_qubit))

---
#### **Phase #2: Bob Receives**
---

<a name="s5"></a>

#### **Step #5: Bob Randomly Chooses Bases**

**Independently**, randomly choose Bob's `num_bits` bases and apply the appropriate gates to the qubits he received, `qubits`. Complete the code below to accomplish this for each qubit.

In [ ]:
bob_bases = choices(# COMPLETE THIS CODE
print('Bob\'s randomly chosen bases: ', bob_bases)

bob_circuit = cirq.Circuit()

for # COMPLETE THIS CODE

  basis_value = # COMPLETE THIS CODE
  basis_gate = # COMPLETE THIS CODE

  qubit = # COMPLETE THIS CODE
  bob_circuit.append(# COMPLETE THIS CODE

#####**Solution**

In [ ]:
bob_bases = choices(['Z', 'X'], k = num_bits)
print('Bob\'s randomly chosen bases: ', bob_bases)

bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

Bob's randomly chosen bases:  ['Z', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z']


<a name="s6"></a>

#### **Step #6: Bob Measures the Qubits**

**Independently**, make a measurement of all of the qubits for Bob. Complete the code below to append this action to `bob_circuit`. Ensure the measure method is given the parameter `key = 'bob key'` so that we can easily retrieve this result later.

In [ ]:
bob_circuit.append(# COMPLETE THIS CODE

print(bob_circuit)

#####**Solution**

In [ ]:
bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

print(bob_circuit)

q0: ────I───M('bob key')───
            │
q1: ────H───M──────────────
            │
q2: ────H───M──────────────
            │
q3: ────H───M──────────────
            │
q4: ────H───M──────────────
            │
q5: ────H───M──────────────
            │
q6: ────H───M──────────────
            │
q7: ────I───M──────────────
            │
q8: ────H───M──────────────
            │
q9: ────H───M──────────────
            │
q10: ───H───M──────────────
            │
q11: ───I───M──────────────
            │
q12: ───I───M──────────────
            │
q13: ───H───M──────────────
            │
q14: ───I───M──────────────
            │
q15: ───I───M──────────────
            │
q16: ───H───M──────────────
            │
q17: ───H───M──────────────
            │
q18: ───H───M──────────────
            │
q19: ───I───M──────────────
            │
q20: ───I───M──────────────
            │
q21: ───H───M──────────────
            │
q22: ───I───M──────────────
            │
q23: ───I───M──────────────
      

<a name="s7"></a>

#### **Step #7: Bob Creates a Key**

**Independently**, create a key for Bob from the mesurement result of each qubit.

<br>

**Run the code below to accomplish this step.**

In [ ]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)

#####**Solution**

In [ ]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 0 0
 0 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0
 1 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 0]


---
#### **Phase #3: Alice and Bob Compare**
---

<a name="s8"></a>

#### **Step #8: Alice and Bob Compare Bases**

**Independently**, compare Alice's and Bob's randomly selected bases. Complete the code below to accomplish this. If they agree on a basis (publicly), then Eve puts her qubit in this basis and measures it.

In [ ]:
final_alice_key = []
final_bob_key = []
final_eve_key = []

for # COMPLETE THIS CODE

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(# COMPLETE THIS CODE
    final_bob_key.append(# COMPLETE THIS CODE

In [ ]:
basis_value = bob_bases[bit]
basis_gate = basis_gates[basis_value]

qubit = eve_qubits[bit]
bb84_circuit.append(basis_gate(qubit))
bb84_circuit.append(cirq.measure(qubit))

sim = cirq.Simulator()
results = sim.run(bb84_circuit)

final_eve_key.append(results.measurements['eve' + str(bit)][0][0])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)

#####**Solution**

In [ ]:
final_alice_key = []
final_bob_key = []
final_eve_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])

In [ ]:
basis_value = bob_bases[bit]
basis_gate = basis_gates[basis_value]

qubit = eve_qubits[bit]
bb84_circuit.append(basis_gate(qubit))
bb84_circuit.append(cirq.measure(qubit))

sim = cirq.Simulator()
results = sim.run(bb84_circuit)

final_eve_key.append(results.measurements['eve' + str(bit)][0][0])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)

NameError: ignored

<a name="s9"></a>

#### **Step #9: Alice and Bob Compare the First Bits in Their Key**

**Independently**, compare the first few bits in Alice's and Bob's key to ensure the protocol was successful.

<br>

**Run the code below to accomplish this.**

In [ ]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')

#####**Solution**

In [ ]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



Eve was listening, we need to use a different channel!


# End of Notebook

---
© 2023 The Coding School, All rights reserved